In [1]:
!pip install opencv-contrib-python==3.4.8.29

     |████████████████████████████████| 34.2MB 88kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [2]:
import os
import cv2
import sys
import numpy as np
from math import cos, sin
import numpy as np
from keras.layers import Average

Using TensorFlow backend.


In [0]:
!mkdir face_detector

In [0]:
  # load our serialized face detector from disk
def draw_results_ssd(detected,input_img,ad,img_w,img_h):
    
    # loop over the detections
    if detected.shape[2]>0:
        for i in range(0, detected.shape[2]):
            confidence = detected[0, 0, i, 2]
            # filter out weak detections
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the face and extract the face ROI
                (h0, w0) = input_img.shape[:2]
                box = detected[0, 0, i, 3:7] * np.array([w0, h0, w0, h0])
                (startX, startY, endX, endY) = box.astype("int")
                # print((startX, startY, endX, endY))
                x1 = startX
                y1 = startY
                w = endX - startX
                h = endY - startY
                
                bounding_box_one = (x1,y1,w,h)
                bounding_box_one=tuple([int(i) for i in bounding_box_one])

                print(bounding_box_one,'--------------------------------------------')

                cv2.rectangle(input_img, (startX, startY), (endX, endY),(0, 255, 0))               
    
    return input_img, bounding_box_one


print("[INFO] loading face detector...")
protoPath = os.path.sep.join(["face_detector", "deploy.prototxt"])
print('yes1')
modelPath = os.path.sep.join(["face_detector",
    "res10_300x300_ssd_iter_140000.caffemodel"])
print('yes2')
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
print('yes3')
# capture video
cap = cv2.VideoCapture('video.mp4')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1024*1)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 768*1)
detected_pre = np.empty((1,1,1))
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 

size = (frame_width, frame_height)
result = cv2.VideoWriter('filename.avi',  
                      cv2.VideoWriter_fourcc(*'MJPG'), 
                      10, size) 

img_idx = 0
detected = '' 
skip_frame = 20 # every 20 frame do 1 detection and network forward propagation
ad = 0.6

while True:
      # get video frame
      ret, input_img = cap.read()
      if ret == False:
          break
      
      img_idx = img_idx + 1
      img_h, img_w, _ = np.shape(input_img)        
      if img_idx==1 or img_idx%skip_frame == 0:
          blob =cv2.dnn.blobFromImage(cv2.resize(input_img, (300, 300)), 1.0,
              (300, 300), (104.0, 177.0, 123.0))
          net.setInput(blob)
          detected = net.forward()
          print(detected.shape,'detected_shape')
  
          if detected_pre.shape[2] > 0 and detected.shape[2] == 0:
                detected = detected_pre
          confident = detected[0, 0, 0, 2]
            # filter out weak detections
          if confident > 0.5:
            input_img, bounding_box_one = draw_results_ssd(detected,input_img,ad,img_w,img_h)

          tracker = cv2.TrackerCSRT_create()
     
          print(type(bounding_box_one),"bounding_box")
          success = tracker.init(input_img, bounding_box_one)
      else:            
        (success, box) = tracker.update(input_img)
        if success:
          (x, y, w, h) = [int(v) for v in box]
          cv2.rectangle(input_img, (x, y), (x + w, y + h),(0, 255, 0),2)
          bounding_box_one = (x, y, w, h)
      result.write(input_img)

        

cap.release()
result.release()

